In [16]:

from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep
import re
import random as rand
from tqdm import tqdm

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [2]:
def get_table(table_text, headers, location):
    table_list = table_text.split("\n")

    split_number = table_list.index('Max Avg Min')-1
    table_final = []
    for i in range(1, split_number+1):
        table_final.append(table_list[i::split_number])

    df_inicial = pd.DataFrame(table_final)
    df_final = pd.DataFrame()
    for j in range(df_inicial.shape[1]):
        df_intermedio = df_inicial[j].str.split(" ", expand=True)
        df_final = pd.concat([df_final, df_intermedio], axis=1)

    df_final.columns = df_final.iloc[0]

    month = df_final.iloc[0,0]
    newcols = [headers[0]]

    for h in headers[1:-1]:
        for i in range(1,4):
            newcols.append(re.sub(r"\(.+\)"," ",h)+df_final.columns[i])

    newcols.append("Precipitations")

    df_final.columns = newcols

    df_final.drop(index = 0, inplace = True)
    df_final.reset_index(drop = True, inplace=True)
    df_final.insert(column="Month", loc=0, value = month)
    df_final.insert(column="Location", loc=0, value = location)
    return df_final


def wait_and_click(drivers, xpath):
    tries = 0
    while True and tries < 5:
        try:
            button = WebDriverWait(drivers, 5).until(EC.presence_of_element_located(("xpath", xpath)))
            sleep(2)
            button.click()
            break
        except:
            tries += 1
            print("Try number", tries)
            continue
    return button


In [14]:
df = pd.read_csv("datos/df_municipio_cat.csv")

municipios = df["Nombre_Municipio"].unique()

municipios_aleatorio = rand.choices(municipios, k = 5)
municipios_aleatorio

['Villaviciosa de Odón',
 'Fuente el Saz de Jarama',
 'Valdilecha',
 'Valdemoro',
 'Navas del Rey']

In [ ]:
df_concat = pd.DataFrame()

for municipio in municipios_aleatorio:
        driver = webdriver.Chrome()

        url_wunder = "https://www.wunderground.com/history"

        driver.maximize_window()

        driver.get(url_wunder)

        search = municipio # Sustituir por municipio

        cookies_frame = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", "#sp_message_iframe_1165301")))
        driver.switch_to.frame(cookies_frame)
        reject_button = wait_and_click(driver, '//*[@id="notice"]/div[3]/div[2]/button')

        sleep(1)

        driver.switch_to.default_content()

        text_box = driver.find_element("xpath", '//*[@id="historySearch"]')
        text_box.send_keys(search)

        WebDriverWait(driver,10).until(EC.presence_of_element_located(("css selector", '#historyForm > search-autocomplete > ul')))
        sleep(1)
        text_box.send_keys(Keys.ENTER)
        view_button = driver.find_element("xpath", '//*[@id="dateSubmit"]')
        view_button.click()

        monthly_button = wait_and_click(drivers= driver, xpath = '//*[@id="inner-content"]/div[2]/div[1]/div[1]/div[1]/div/lib-link-selector/div/div/div/a[3]')
        sleep(1.5)
        df_month = pd.DataFrame()
        for month_index in range(9,-1,-1):
            table = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table')))
            table_html = table.get_attribute("innerHTML")
            table_text = table.text

            soup_table = BeautifulSoup(table_html, "html.parser")

            headers = [h.text for h in soup_table.find("tr").findAll("td")]
            df_municipio = get_table(table_text, headers, municipio)
            df_month = pd.concat([df_month,df_municipio])
            if month_index == 0:
                  break
            wait_and_click(drivers=driver, xpath='//*[@id="monthSelection"]')
            wait_and_click(drivers=driver, xpath=f'//*[@id="monthSelection"]/option[{month_index}]')
            wait_and_click(drivers=driver, xpath='//*[@id="dateSubmit"]')
        
        driver.quit()
        df_concat = pd.concat([df_concat,df_month])
        


In [6]:
df_concat

,Location,Month,Time,Temperature Max,Temperature Avg,Temperature Min,Dew Point Max,Dew Point Avg,Dew Point Min,Humidity Max,Humidity Avg,Humidity Min,Wind Speed Max,Wind Speed Avg,Wind Speed Min,Pressure Max,Pressure Avg,Pressure Min,Precipitations
0,Daganzo de Arriba,Oct,1,84,66.8,50,55,45.2,41,72,48.8,23,10,3.6,0,28.0,27.9,27.9,0.00
1,Daganzo de Arriba,Oct,2,79,71.2,63,63,57.0,52,78,62.0,47,18,8.8,0,27.9,27.8,27.7,0.00
2,Daganzo de Arriba,Oct,3,77,67.6,61,59,49.6,37,88,55.5,24,16,5.6,0,27.9,27.8,27.7,0.00
3,Daganzo de Arriba,Oct,4,77,63.9,50,50,46.1,43,82,53.9,36,8,2.3,0,27.9,27.9,27.8,0.00
4,Daganzo de Arriba,Oct,5,81,65.5,50,59,51.0,45,87,61.8,39,17,5.9,0,27.9,27.9,27.8,0.00
5,Daganzo de Arriba,Oct,6,75,69.7,64,59,56.7,55,73,64.1,50,17,12.7,1,27.8,27.8,27.8,0.00
6,Daganzo de Arriba,Oct,7,75,63.6,57,57,53.6,48,94,71.8,38,24,6.9,0,27.8,27.7,27.6,0.00
7,Daganzo de Arriba,Oct,8,64,60.0,55,57,54.6,52,88,82.0,68,17,11.4,3,27.8,27.8,27.7,0.00
8,Daganzo de Arriba,Oct,9,70,63.0,59,63,57.3,50,94,81.4,63,23,12.2,3,27.8,27.6,27.5,0.00
9,Daganzo de Arriba,Oct,10,68,60.4,52,50,43.9,41,76,55.4,37,20,7.3,0,27.9,27.8,27.7,0.00


In [7]:
# tables = [s.text for s in soup_table.findAll("table")]
# monthday = tables[0]
# precipitation = tables[-1]
# pd.concat([pd.Series(monthday.split()), pd.Series(precipitation.split())], axis = 1)
# values = tables[1:-1]
# # pd.DataFrame(row)[0].str.split(expand=True)

# df_final_values = pd.DataFrame()
# for i in range(len(values)):
#     maximum = values[i].split()[::3]
#     average = values[i].split()[1::3]
#     minimum = values[i].split()[2::3]
#     df_values = pd.DataFrame(dict(Max = maximum, Avg = average, Min = minimum))
#     df_final_values = pd.concat([df_final_values, df_values], axis = 1)

# pd.concat([df_final_values, pd.Series(precipitation)], axis = 0)

In [25]:
def tables2(tabletext, tablehtml):
    soup2 = BeautifulSoup(tablehtml, "html.parser")
    headers2 = [h.text for h in soup2.find("thead").findAll("th")]

    amounts = ["High", "Avg", "Low"]

    new_headers = []
    new_headers.append("Date")
    for h in headers2[1:5]:
        for a in amounts: 
            new_headers.append(a +" "+ h)

    for a in ["High", "Low"]:
        new_headers.append(a +" "+ headers2[5])
    new_headers.append(headers2[-1])
    table2 = tabletext.replace("°F ", "").replace("% ", "").replace("mph ", "").replace("in", "").split()
    df_table2 = pd.DataFrame(np.reshape(table2, (int(len(table2)/16),16)))
    df_table2.columns = new_headers

    return df_table2

In [31]:
df_final = pd.DataFrame()
dict_codes = dict()
for municipio in tqdm(municipios_aleatorio):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url = "https://www.wunderground.com")
    cookies_frame = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", "#sp_message_iframe_1165301")))
    driver.switch_to.frame(cookies_frame)
    reject_button = wait_and_click(driver, '//*[@id="notice"]/div[3]/div[2]/button')
    sleep(1.5)
    driver.switch_to.default_content()
    search_box = wait_and_click(driver, xpath = '/html/body/app-root/app-home-page/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[1]/div[2]/lib-search/div/div/div/input')
    search_box.send_keys(municipio+", Madrid")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="wuForm"]/search-autocomplete/ul')))
    wait_and_click(driver, xpath='//*[@id="wuForm"]/search-autocomplete/ul/li[2]/a/span[1]')

    wait_and_click(driver, xpath='//*[@id="inner-content"]/div[2]/lib-city-header/div[1]/div/div/a[1]')

    code = driver.find_element("xpath", '//*[@id="inner-content"]/div[1]/app-dashboard-header/div[2]/div/div[2]/h1').text.split("-")[1].strip()
    df_municipio = pd.DataFrame()
    for monthid in range(10,-1,-1):
        if monthid==0:
            break
        url_getres = f"https://www.wunderground.com/dashboard/pws/{code}/table/2024-{monthid}-16/2024-{monthid}-16/monthly"
        print(url_getres)

        driver.get(url_getres)
        driver.execute_script("window.scrollTo(0, 1600)")
        sleep(1)
        table = driver.find_element("xpath", '//*[@id="main-page-content"]/div/div/div/lib-history/div[2]/lib-history-table/div/div/div/table/tbody').text
        table_html2 = driver.find_element("xpath", '//*[@id="main-page-content"]/div/div/div/lib-history/div[2]/lib-history-table/div/div/div/table').get_attribute("innerHTML")

        df_month = tables2(table,table_html2)
        df_municipio = pd.concat([df_municipio, df_month])
    df_municipio.insert(column="Municipio", loc = 0, value = municipio)
    df_final = pd.concat([df_final, df_municipio])

    driver.quit()


  0%|          | 0/5 [00:00<?, ?it/s]

https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-10-16/2024-10-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-9-16/2024-9-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-8-16/2024-8-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-7-16/2024-7-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-6-16/2024-6-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-5-16/2024-5-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-4-16/2024-4-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-3-16/2024-3-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-2-16/2024-2-16/monthly
https://www.wunderground.com/dashboard/pws/IVILLA777/table/2024-1-16/2024-1-16/monthly


 20%|██        | 1/5 [00:55<03:42, 55.69s/it]

https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-10-16/2024-10-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-9-16/2024-9-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-8-16/2024-8-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-7-16/2024-7-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-6-16/2024-6-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-5-16/2024-5-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-4-16/2024-4-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-3-16/2024-3-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-2-16/2024-2-16/monthly
https://www.wunderground.com/dashboard/pws/IFUENT45/table/2024-1-16/2024-1-16/monthly


 40%|████      | 2/5 [02:49<04:30, 90.08s/it]

...

 60%|██████    | 3/5 [04:43<03:21, 100.84s/it]

https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-10-16/2024-10-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-9-16/2024-9-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-8-16/2024-8-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-7-16/2024-7-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-6-16/2024-6-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-5-16/2024-5-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-4-16/2024-4-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-3-16/2024-3-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-2-16/2024-2-16/monthly
https://www.wunderground.com/dashboard/pws/IVALDE265/table/2024-1-16/2024-1-16/monthly


 80%|████████  | 4/5 [06:44<01:48, 108.62s/it]

https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-10-16/2024-10-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-9-16/2024-9-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-8-16/2024-8-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-7-16/2024-7-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-6-16/2024-6-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-5-16/2024-5-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-4-16/2024-4-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-3-16/2024-3-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-2-16/2024-2-16/monthly
https://www.wunderground.com/dashboard/pws/INAVAS9/table/2024-1-16/2024-1-16/monthly


100%|██████████| 5/5 [08:45<00:00, 105.05s/it]


In [33]:
df_final.to_csv("datos/df_final.csv")

In [34]:
df_final.head()

,Municipio,Date,High Temperature,Avg Temperature,Low Temperature,High Dew Point,Avg Dew Point,Low Dew Point,High Humidity,Avg Humidity,Low Humidity,High Speed,Avg Speed,Low Speed,High Pressure,Low Pressure,Sum
0,Villaviciosa de Odón,10/1/2024,84.0,65.7,49.6,50.9,43.0,36.1,68,46,25,10.7,1.7,0.0,29.95,29.82,0.00
1,Villaviciosa de Odón,10/2/2024,76.6,68.1,60.1,61.7,56.9,48.7,82,68,51,20.1,4.3,0.0,29.88,29.70,0.01
2,Villaviciosa de Odón,10/3/2024,75.9,68.4,61.5,59.0,52.3,43.2,76,58,34,18.1,5.5,0.0,29.85,29.72,0.00
3,Villaviciosa de Odón,10/4/2024,77.7,64.6,50.7,54.5,50.0,44.4,88,61,40,10.7,1.6,0.0,29.89,29.80,0.03
4,Villaviciosa de Odón,10/5/2024,78.6,64.1,49.8,59.7,52.4,45.5,86,67,43,14.8,3.1,0.0,29.87,29.78,0.00
